# Práctica 2 Parte 1

## Descripción

La primera parte consiste en probar el método implementado en la práctica 1 con diferentes experimentos, en este caso lo probaremos con distintas redes. Los ejemplos auxiliares serán utilizados para encontrar errores o áreas de oportunidad de nuestro algoritmo.
Consideramos que el único parámetro de nuestro algoritmo es la matriz o grafo en el que trabajaremos para correr Ford-Fulkerson, ya que nuestra función lee la matriz e inmediatamente reconoce los nodos y los arcos para poder calcular el flujo máximo. Por lo que decidimos probar nuestro método con algunos grafos distintos.

Para consultar la información de nuestro paquete, vea: [reporte parte 2 práctica 1](https://github.com/optimizacion-2-2022-gh-classroom/practica-1-segunda-parte-LuzVerde23/blob/main/reporte_equipo_2_parte_2_practica_1.ipynb).


In [1]:
import time
import pandas as pd
import networkx as nx
import random
#from MaxFlowAeiu.MaxFlowAeiu import MaxFlowAeiu
from MaxFlowAeiu import MaxFlowAeiu

## Ejemplo 1
Para comenzar usaremos el ejemplo de la práctica uno

In [2]:
url_d = "https://raw.githubusercontent.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/main/BD/d.csv"
d = pd.read_csv(url_d,header=None)

In [3]:
arreglo = d.values.tolist()

In [4]:
MF=MaxFlowAeiu(arreglo)

In [5]:
%time
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))

CPU times: user 50 µs, sys: 0 ns, total: 50 µs
Wall time: 60.1 µs
The maximum flow in this network is 1480.0


Al ejecutar este ejemplo en el ambiente virtual (instancia de AWS) y monitorear el flujo de trabajo, obtenemos el siguiente resultado:
<p align = "center">
    <img src="images/ejemplo1.png" width="1329" height="911" />

# Ejemplo 2
Seguimos con un ejemplo de menor escala que trata sobre el transporte de personas en una empresa de autobúses

In [6]:
d2 = [[0,188,240,160,0,0,0,0,0,0,0,0,0],
                    [0,0,0,0,170,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,210,180,0,0,0,0,0],
                    [0,0,0,0,0,140,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,130,90,0,0],
                    [0,0,0,0,0,0,0,0,130,0,0,0,0],
                    [0,0,0,0,0,0,0,0,150,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,160,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,210],
                    [0,0,0,0,0,0,0,0,0,0,0,0,110],
                    [0,0,0,0,0,0,0,0,0,0,0,0,80],
                    [0,0,0,0,0,0,0,0,0,0,0,0,170],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0]]


In [7]:
arreglo2 = d2

In [8]:
MF2=MaxFlowAeiu(arreglo2)

In [9]:
%time
print("The maximum flow in this network is {}".format(MF2.ford_fulkerson()))

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 14.5 µs
The maximum flow in this network is 540


## Ejemplo 3
Ahora probaremos con una matriz más grande sobre el transporte de pasajeros en aerolíneas en Estados Unidos

In [10]:
d3=pd.read_csv("https://raw.githubusercontent.com/sid-7/Airline_Maximum_Flow/master/flights.csv")
d3=d3.drop(['Unnamed: 0', 'Depature', 'Arrival'], axis=1)
d3=d3.groupby(['Source', 'Destination'], as_index=False)['capacity'].sum()
d3['capacity'] = pd.to_numeric(d3['capacity'], errors='coerce')
new_row = pd.DataFrame([['JFK','JFK', 0], ['LAX','LAX', 0]],
                   columns=['Source', 'Destination', 'capacity'])
d3 = pd.concat([d3, new_row])
d3 = d3.pivot(index="Source", columns="Destination", values="capacity").fillna(0)
column_to_move1 = d3.pop("LAX")
column_to_move2 = d3.pop("JFK")
d3.insert(0, "LAX", column_to_move1)
d3.insert(9, "JFK", column_to_move2)
t1 = ['JFK','LAX']
t2 = ['JFK']
t3 = ['LAX']

a = d3.loc[[i for i in d3.index if i not in t1], :]
b = d3.loc[t2, :]
c = d3.loc[t3, :]

d3=pd.concat([c, a, b])

In [11]:
arreglo3 = d3.values.tolist()

In [12]:
MF3=MaxFlowAeiu(arreglo3)

In [13]:
%time
print("The maximum flow in this network is {}".format(MF3.ford_fulkerson()))

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 16 µs
The maximum flow in this network is 6427.0


## Ejemplo 4
Ahora usaremos una matriz mucho más grande

In [14]:
# Especificar el tamaño de la red n = número de nodos, m = número de ramas
n = 1000
m = 5000
network = nx.gnm_random_graph(n, m, directed=True)
nodos = list(network.nodes)
ramas = list(network.edges)
# Crear arreglos de: nodo de envío, nodo de recepción y capacidad del enlace
env = []
rec = []
cap = []
for i in range(len(ramas)):
    env.append(ramas[i][0])
    rec.append(ramas[i][1])
    cap.append(random.randint(20,100)) # La capacidad del enlace se determinade forma aleatoria con valores entre 20 y 100

In [15]:
# Se crea el data frame o matriz de incidencias de los nodos. Se asegura que el nodo fuente sea el 0 y solo tenga ramas saliendo de él y que el último nodo solo tenga ramas entrando (sumidero)
d4 = pd.DataFrame(0, index=list(range(0,len(nodos))), columns=list(range(0,len(nodos))))
aux = 0
for n in env:
    if rec[aux] == 0:
        d4.iloc[rec[aux],n] = cap[aux]
    elif n == max(nodos):
        d4.iloc[rec[aux],n] = cap[aux]
    else:
        d4.iloc[n,rec[aux]] = cap[aux]
    aux += 1

In [16]:
arreglo4 = d4.values.tolist()

In [17]:
MF4=MaxFlowAeiu(arreglo4)

In [18]:
%time
print("The maximum flow in this network is {}".format(MF4.ford_fulkerson()))

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 13.8 µs
The maximum flow in this network is 567


## Reporte de resultados

La realización de estos 4 ejercicios se hizo en un ambiente virtual (instancia de AWS) con las herramientas de Kale, Minikube y Kubeflow. En la siguiente imagen se muestra el pipeline ejecutado con los cuatro ejemplos corriendo de manera satisfactoria:

<p align = "center">
    <img src="images/LAHM_kale_ec2-44-204-141-137.png" width="1329" height="911" />

Adicionalmente se realizó la misma ejecución de los 4 ejercicios, pero en el ambiente virtual generado para este repositorio desde la funcionalidad de _Binder_. En la siguiente tabla se muestran los resultados obtenidos en ambas ejecuciones:

In [19]:
results = {
    'Experimento' : [1, 1, 2, 2, 3, 3, 4, 4],
    'Tamaño_Red': ["Mediana", "Mediana", "Pequeña", "Pequeña", "Pequeña", "Pequeña", "Grande", "Grande" ],
    'Nodos': [44, 44, 13, 13, 10, 10, 1000, 1000],
    'Ramas': [61, 61, 16, 16, 72, 72, 5000, 5000],
    'Flujo_Max': [1480, 1480, 540, 540, 6427, 6427, 643, 513],
    'Exitoso': ['Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si'],
    'CPU_Time_(ms)': [4.65, 7e-3, 3.37, 6e-3, 7.51, 7e-3, 26e3, 6e-3],
    'Wall_Time_(ms)': [7.68, 14.1e-3, 3.15, 11.9e-3, 5.62, 12.9e-3, 26e3, 12.2e-3],
    'Fecha': ["22-04-2022", "23-04-2022", "22-04-2022", "23-04-2022", "22-04-2022", "23-04-2022", "22-04-2022", "23-04-2022"],
    'Instancia': ["AWS", "Binder", "AWS", "Binder", "AWS", "Binder", "AWS", "Binder"]
}
r = pd.DataFrame(results)
r

,Experimento,Tamaño_Red,Nodos,Ramas,Flujo_Max,Exitoso,CPU_Time_(ms),Wall_Time_(ms),Fecha,Instancia
0,1,Mediana,44,61,1480,Si,4.650,7.6800,22-04-2022,AWS
1,1,Mediana,44,61,1480,Si,0.007,0.0141,23-04-2022,Binder
2,2,Pequeña,13,16,540,Si,3.370,3.1500,22-04-2022,AWS
3,2,Pequeña,13,16,540,Si,0.006,0.0119,23-04-2022,Binder
4,3,Pequeña,10,72,6427,Si,7.510,5.6200,22-04-2022,AWS
5,3,Pequeña,10,72,6427,Si,0.007,0.0129,23-04-2022,Binder
6,4,Grande,1000,5000,643,Si,26000.000,26000.0000,22-04-2022,AWS
7,4,Grande,1000,5000,513,Si,0.006,0.0122,23-04-2022,Binder


## Reimplementación

Como se menciona en la primera parte de este reporte (Descripción), las pruebas que se realizaron para comprobar el correcto funcionamiento del algoritmo implementado consisten únicamente en comprobar con diferentes redes, encontrando que en todos los casos se logró encontrar el flujo máximo. En este sentido, no se vió la necesidad de hacer una reimplementación del método en cuanto al algoritmo. Sin embargo, para ejemplificar la utilización de las herramientas que se han revisado e implementado en estas 2 prácticas, se adiciona una función adicional a nuestro algoritmo _MaxFlowAeiu_ que cuenta el número de nodos y ramas que tiene la red. Esta nueva funcionalidad se actualiza en la documentación y se vuelve a ejecutar el pipeline completo para validar que sigue ejecutandose de manera exitosa y que la nueva implementación funciona adecuadamente. En la imagen siguiente se muestra el resultado de esta reimplementación:

Veamos la información del primer ejemplo.

In [20]:
%time
MF.infoMF()

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 16.5 µs
nodos 44
ramas 84


Podemos ver que nuestro ejemplo 1 tiene 44 nodos y 84 arcos. Consideramos que esta información es una aportación útil al método ya que pueden existir ocasiones en que querramos ver si la información con la que contamos correrá de forma rápida, como es en este caso, o se tardará más tiempo en caso de tener una matriz muy conectada, como lo es el caso del ejemplo 4:

In [21]:
%time
MF4.infoMF()

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.3 µs
nodos 1000
ramas 5073


<p align = "center">
    <img src="images/LAHM_kale_ec2-44-204-141-137.png" width="1329" height="911" />

## Conclusiones

En el experimento ejecutado en la instancia de AWS vemos que el desempeño de nuestro algoritmo da buenos resultados en cuanto a tiempo de ejecución. En los primeros 3 ejemplos fue casi imperceptible, del orden de los milisegundos (ms), para el ejemplo con una red de 1000 nodos con 5000 arcos tomó 26 segundos en obtener el flujo máximo, lo cual sigue siendo bueno. En todos los casos se obtuvo una solución de flujo máximo, por lo tanto fueron ejecuciones exitosas.

Notamos una diferencia notoria en los timepos de ejecución del experimento en la instancia creada desde Binder, en la cual los tiempos de ejecución se reducen notablemente, llegando al orden de los microsegundos, incluso para la red más grande que se trabajó (1000 nodos y 5000 ramas). Cabe resaltar que los resultados que obtiene el método, tanto en el experimento ejecutado en AWS como en Binder, son iguales para las redes que son definidas (ejemplos 1 a 3), pero se obtiene un resultado diferente para la red "grande", ya que se genera de manera aleatoria y por lo tanto la solución va a cambiar dependiendo de la semilla que se tenga en cada instancia.

A futuro, consideramos que nos hizo falta poder hacer uso de la librería `numpy` (como lo hicimos en la práctica 1 y en nuestra imagen de manera local), pero en el ambiente de `kale`siempre nos marcó un error cuando intentábamos usar algunos módulos de dicha librería. Creemos que es un tema de las versiones, así que esperamos poder encontrar la solución para la parte 2 de la práctica. Ejemplo:


<p align = "center">
    <img src="https://files.gitter.im/620bf57c6da0373984906852/M3CK/Screen-Shot-2022-04-21-at-11.23.44.png" width="1329" height="911" />


## Referencias
* [1] [Palacios E. (2022) Libro de Optimización](https://itam-ds.github.io/analisis-numerico-computo-cientifico/4.optimizacion_en_redes_y_prog_lineal/4.2/Definiciones_generales_de_flujo_en_redes.html)
* [2] [Dumora c. el all. Data Oriented Algorithm for Real Time Estimation of Flow Rates and Flow Directions in Water Distribution Network](https://arxiv.org/pdf/1807.10147.pdf)
* [3] [Max Flow Problem Introduction](https://www.geeksforgeeks.org/max-flow-problem-introduction/)
* [4] [Ford-Fulkerson Algorithm](https://www.programiz.com/dsa/ford-fulkerson-algorithm)
* [5] [Algoritmo de Ford-Fulkerson - Ford–Fulkerson algorithm](https://upwikies.top/wiki/Ford%e2%80%93Fulkerson_algorithm)
* [6] [Oviedo J. (2008) Algoritmo de Ford-Fulkerson Mejorado](http://www.ptolomeo.unam.mx:8080/jspui/bitstream/132.248.52.100/2387/1/gonzalezoviedo.pdf)
* [7] [Building a Smarter (and Cheaper) School Bus System: How a Boston-MIT Partnership Led to New Routes That Are 20% More Efficient and Saved the District $5 Million](https://www.the74million.org/article/building-a-smarter-and-cheaper-school-bus-system-how-a-boston-mit-partnership-led-to-new-routes-that-are-20-more-efficient-use-400-fewer-buses-save-5-million/)
* [8] [Optimazation examples](https://vitalflux.com/convex-optimization-explained-concepts-examples/)
* [9] [Breadth First Search or BFS for a Graph](https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/)
* [10] [Create your own Python package and publish it into PyPI](https://towardsdatascience.com/create-your-own-python-package-and-publish-it-into-pypi-9306a29bc116)